In [ ]:
#import dependency
from bs4 import BeautifulSoup as bs
import numpy as np
import time
import pandas as pd
from tqdm import tqdm
import re
import json

#selenium stuff
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# **STATIC VARIABLE**

In [ ]:
sheet_name= "e_bike"
link_page = "https://www.bike24.com/e-bikes.html?menu=1000,314"
file_name = "output.xlsx"

wait_min = 5
wait_max = 10

# **GET ALL EBIKE LIST**

In [ ]:
page = []
bike_name = []
bike_id = []
bike_price = []
bike_brand = []
bike_color = []
category = []

#open browser
driver = uc.Chrome()

#set window size
driver.set_window_size(1920, 1080)
driver.maximize_window()

#check how many pagination the page has
driver.get(f'{link_page}')
pagination = bs(driver.page_source, 'html.parser').find_all('a', attrs={'class':'page-pagination-item'})
pagination = int(pagination[-1].get('data-pager-idx'))


#iterate throught pagination
for i in tqdm(range(1,pagination+1)):

    driver.get(f'{link_page}&page={i}')

    #wait until cookies modal ready to clickable
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)"))
        )

    except:
        pass
    
    #if okay, click the modal
    else:
        driver.find_element(By.CSS_SELECTOR, value='#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)').click()

    finally:
        time.sleep(np.random.randint(wait_min, wait_max))
        driver.execute_script("window.scrollTo(0, (document.body.scrollHeight));")
        time.sleep(np.random.randint(wait_min, wait_max))
        driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)/5);")
        time.sleep(np.random.randint(wait_min, wait_max))

        #pick page source code
        source = driver.page_source
        source = bs(source, 'html.parser')

        #get item list
        items = source.find_all('a', attrs={'class':'product-tile__anchor gtm-link-root'})

        #iterate through items
        for item in items:

            #get datalayer enhance ecommerce data
            temp = item.get('data-gtm')
            temp = json.loads(temp)
            temp = temp['ecommerce']['click']['products'][0]
            pageTemp = str('https://www.bike24.com' + str(item.get('href')))
            page.append(pageTemp)
            bike_name.append(temp['name'])
            bike_id.append(temp['id'])
            bike_price.append(temp['price'])
            bike_brand.append(temp['brand'])
            bike_color.append(temp['variant'])
            category.append(temp['category'])

        #random sleep
        time.sleep(np.random.randint(wait_min, wait_max))

        #Create dataframe to save the scraped item
        df = pd.DataFrame({ 
                            'Page Link': page,
                            'Bike Id': bike_id,
                            'Bike Brand' : bike_brand,
                            'Bike Name': bike_name,
                            'Bike Price €': bike_price,
                            'Bike Color' : bike_color,
                            'Category': category
                        })
        #update column data type
        df['Bike Price €'] = df['Bike Price €'].astype('float')
        df['Bike Id'] = df['Bike Id'].astype('int')

        #clean page link
        df['Page Link'] = df['Page Link'].apply(lambda x: x.split('?')[0])
        
        #if file is already there, then append the data
        try:
            with pd.ExcelWriter(f'{file_name}', mode='a', if_sheet_exists='overlay') as writer:  
                df.to_excel(writer, sheet_name=f'{sheet_name}_list', index=False)
        
        #else, create new file
        except:
            with pd.ExcelWriter(f'{file_name}', mode='w') as writer:  
                df.to_excel(writer, sheet_name=f'{sheet_name}_list', index=False)
            
#when done, close the driver
driver.close()

# **FILE UPDATE CHECKER, RUN THIS TO CONTINUE THE PAUSED PROCESS**

In [ ]:
#open file
df = pd.read_excel(file_name, sheet_name=f'{sheet_name}_list')
df_temp = pd.read_excel(file_name, sheet_name=f'{sheet_name}_list_detail')

vlist = df['Page Link'].values
dlist = df_temp['Page Link'].values
filterTable = [link for link in vlist if link not in dlist]

df_detail = df.loc[df['Page Link'].isin(filterTable)]

# **GET EBIKE DETAIL**

In [ ]:
#create empty dataframe for save all results
all_detail_table = df_temp.copy()

#create empty dataframe for fail result
fail_table = pd.DataFrame(columns=df_detail.columns)

#create empty dataframe for not available result
not_available_table = pd.DataFrame(columns=df_detail.columns)


#open browser
driver = uc.Chrome()

#set window size
driver.set_window_size(1920, 1080)
driver.maximize_window()

#iterate throught rows
for i, r in tqdm(df_detail.iterrows()):

    #open page link
    driver.get(r['Page Link'])
    time.sleep(np.random.randint(wait_min, wait_max))

    #wait until cookies modal ready to clickable
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)'))
        )
    
    except:
        pass
    
    #if okay, click the modal
    else:
        driver.find_element(By.CSS_SELECTOR, value='#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)').click()

    finally:

        #wait until fact sheet is ready to clickable
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__list-wrapper > ul > li.product-detail-tab-nav__list-item.product-detail-tab-nav__list-item--3 > label'))
            )
        
        except:
            
            #check if item are currently not available
            try:
                not_available = driver.find_element(by=By.CSS_SELECTOR, value='body > div.store-page-wrapper > div > div.product-not-available > div.message > div > div')

            #if available, keep going
            except:
                pass
            
            #if not, list it into sheet
            else:
                not_available_temp_table = pd.DataFrame([{
                                        'Page Link': r['Page Link'],
                                        'Bike Id': r['Bike Id'],
                                        'Bike Brand': r['Bike Brand'],
                                        'Bike Name': r['Bike Name'],
                                        'Bike Price €': r['Bike Price €'],
                                        'Bike Color': r['Bike Color'],
                                        'Category': r['Category']
                                        }])

                not_available_table = pd.concat([not_available_table, not_available_temp_table])
                
                with pd.ExcelWriter(f'{file_name}', mode='a', if_sheet_exists='overlay') as writer:
                    not_available_temp_table.to_excel(writer, sheet_name=f'{sheet_name}_not_available_list', index=False)            

                
                #continue to next loop
                continue

        finally:
            #simple fact sheet getter algorithm
            time.sleep(np.random.randint(wait_min, wait_max))
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight));")
            time.sleep(np.random.randint(wait_min, wait_max))
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)/5);")
            time.sleep(np.random.randint(wait_min, wait_max))

            #try to click the fact sheet menu
            try:
                driver.find_element(By.CSS_SELECTOR, value='#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__list-wrapper > ul > li.product-detail-tab-nav__list-item.product-detail-tab-nav__list-item--3 > label ').click()
            
            #if fail, list it into sheet
            except:
                fail_temp_table = pd.DataFrame([{
                                            'Page Link': r['Page Link'],
                                            'Bike Id': r['Bike Id'],
                                            'Bike Brand': r['Bike Brand'],
                                            'Bike Name': r['Bike Name'],
                                            'Bike Price €': r['Bike Price €'],
                                            'Bike Color': r['Bike Color'],
                                            'Category': r['Category']
                                            }])

                #remember that fail item must not same itemwith not available item
                if fail_temp_table['Bike Id'].values  not in not_available_table['Bike Id'].values:                                            
                    fail_table = pd.concat([fail_table, fail_temp_table])

                    with pd.ExcelWriter(f'{file_name}', mode='a', if_sheet_exists='overlay') as writer:
                        fail_table.to_excel(writer, sheet_name=f'{sheet_name}_failed_list', index=False)

            #if success click fact sheet, run processing algorithm
            else:
                #pick page source for parsing
                source = driver.page_source
                source = bs(source, 'html.parser')

                #pick data from FACT SHEET BUTTON
                source = source.find('table', attrs={'class':'product-detail-data-sheet__table'})

                #pick all column from table
                table_column = source.find_all('td', attrs={'class': 'product-detail-data-sheet__cell product-detail-data-sheet__cell--title'})

                #clean column from any symbol & make it uppercase
                table_column = [re.sub(r':', '', item.text) for item in table_column]

                #append add on data
                table_column.append('Page Link')

                #pick all value from table
                table_value = source.find_all('td', attrs={'class': 'product-detail-data-sheet__cell product-detail-data-sheet__cell--value'})
                table_value = [item.text for item in table_value]

                #append page link into table_value
                table_value.append(r['Page Link'])

                #create new table dataframe
                detail_table = dict(zip(table_column, table_value))
                detail_table = pd.DataFrame([detail_table])

                #append current value of table into all detail  table
                all_detail_table = pd.concat([all_detail_table, detail_table], ignore_index=True)

                #input data into excel
                with pd.ExcelWriter(f'{file_name}', mode='a', if_sheet_exists='overlay') as writer:
                    all_detail_table.to_excel(writer, sheet_name=f'{sheet_name}_list_detail', index=False)



            time.sleep(np.random.randint(wait_min, wait_max))
driver.close()

# **FINALIZE THE DATA INTO ASSIGNTMENT FORMAT**

In [ ]:
#open file
df = pd.read_excel(file_name, sheet_name=f'{sheet_name}_list')
df_temp = pd.read_excel(file_name, sheet_name=f'{sheet_name}_list_detail')

#join file
final_df = pd.merge(df, df_temp, on=["Page Link"], how="inner")

#drop
final_df.drop(['Page Link', 'Bike Id', 'Bike Brand', 'Bike Color', 'Category'], inplace=True ,axis=1)

#input data into excel
with pd.ExcelWriter(f'{file_name}', mode='a', if_sheet_exists='overlay') as writer:
    final_df.to_excel(writer, sheet_name=f'{sheet_name}_FINAL_RESULT', index=False)
